In [1]:
import pandas as pd
import utils
import numpy as np
from itertools import combinations
from fuzzywuzzy import fuzz
from sheet_loader.loader import get_worksheet_as_df

# Load data

In [2]:
#utils.generate_csv()
bbt = utils.load_sheet("budget_by_type")
bt = utils.load_sheet("budget_type")
rt = utils.load_sheet("resource_type")
rbt = utils.load_sheet("resource")

In [3]:
rt = get_worksheet_as_df("resource_type")
rbt = get_worksheet_as_df("resource")

In [4]:
expenses = utils.merge(
    hierarchy_df=bt,
    hierarchy_df_on="id",
    values_df=bbt,
    values_df_on="type",
    hierarchy_suffix="_bt",
    values_suffix="_bbt",
    drop_cols=[
        "name_fr",
        "name_en",
        "name_ar",
        "name",
        "description",
        "id_bt",
        "parent_id",
        "type",
        "id_bbt",
        "organization",
    ],
)
revenues = utils.merge(
    hierarchy_df=rt,
    hierarchy_df_on="id",
    values_df=rbt,
    values_df_on="type",
    hierarchy_suffix="_rt",
    values_suffix="_rbt",
    drop_cols=[
        "name_fr",
        "name_en",
        "name_ar",
        "description",
        "description_fr",
        "description_ar",
        "description_en",
        "level",
        "id_rt",
        "parent_id",
        "id_rbt",
    ],
)

# Expenses

## All ministries have top level budget type

Check the number of unique ministries per year, compare it to the number of ministries with a top level budget type

In [5]:
for year in expenses.year.unique():
    print(year)
    for i in utils.top_level_budget(expenses, year):
        if i not in ['الدولة', 'الصندوق العام للتعويـــــض']:
            print(i)

2015
2016
2017
2018
2019


In [6]:
for year in expenses.year.unique():
    print(year)
    orgs = expenses[expenses.year == year].organization_name.unique()
    combs = combinations(orgs, 2)
    for comb in combs:
        ratio = fuzz.token_set_ratio(*comb)
        if 90 < ratio < 100:
            print(comb)

2015
('وزارة التعليم العالي والبحث العلمي وتكنولوجيا المعلومات والاتصال : جزء التعليم العالي و البحث العلمي', 'وزارة التعليم العالي والبحث العلمي وتكنولوجيا المعلومات والاتصال : جزء تكنولوجيا المعلومات و الإتصال')
2016
2017
2018
2019


## State budget must equal the sum of ministries' budgets and state-level expenses

For each year, the sum of ministries budgets and نفقات طارئة و غير موزعة and الدين العمومي must equal ميزانية الدولة

In [7]:
year = 2019

In [8]:
ssb = utils.summed_state_budget(expenses, year)
print(ssb)
print(np.round(sum(ssb.values(), 3)))

{'sum_ministries': 30689.1, 'public_debt': 9.307, 'imprev': 743.9}
31445.0


In [9]:
utils.state_budget(expenses, year)

40741.0

In [10]:
records = []
for year in expenses.year.unique():
    ssb = utils.summed_state_budget(expenses, year)
    sum_ssb = np.round(sum(ssb.values(), 3))
    expected_budget = utils.state_budget(expenses, year)
    ssb["year"] = year
    ssb["expected_budget"] = expected_budget
    ssb["sum_details"] = sum_ssb
    ssb["gap"] = np.round(expected_budget - sum_ssb, 4)
    records.append(ssb)
pd.DataFrame(records).set_index("year")

,expected_budget,gap,imprev,public_debt,sum_details,sum_ministries
year,,,,,,
2015,28900.0,-1388797.0,734.976,5130.000,1417697.0,1411829.378
2016,29150.0,-3.0,398.036,5130.000,29153.0,23621.974
2017,32200.0,-3.0,920.330,5825.000,32203.0,25454.670
2018,35851.0,-3.0,532.620,7972.000,35854.0,27346.380
2019,40741.0,9296.0,743.900,9.307,31445.0,30689.100


In [11]:
expenses[
    (expenses.year == year)
    & (expenses.organization_name == "الدولة")
    & (expenses.extra == 0)
]

,parent_name,value,extra,year,organization_name,budget_type_name
1582,NaN,40741.000,0.0,2019,الدولة,ميزانية الدولة
2046,NaN,743.900,0.0,2019,الدولة,نفقات طارئة و غير موزعة
2051,نفقات طارئة و غير موزعة,493.800,0.0,2019,الدولة,نفقات طارئة و غير موزعة : تصرف
2056,نفقات طارئة و غير موزعة,250.100,0.0,2019,الدولة,نفقات طارئة و غير موزعة : تنمية
2061,ميزانية الدولة,9.307,0.0,2019,الدولة,الدين العمومي
2068,الدين العمومي,3.137,0.0,2019,الدولة,فوائد الدين العمومي
2075,الدين العمومي,6.170,0.0,2019,الدولة,تسديد أصل الدين


In [11]:
gap = utils.budget_gap(expenses)
gap[(gap.gap != 0) & (gap.double != 2)].set_index(["year", "organization_name"])

,,extra,value_agg,parent_name_typed,value_typed,budget_type_name,gap,double
year,organization_name,,,,,,,
2015,وزارة التنمية و التعاون الدولي,0.0,802.012,نفقات التنمية,416.006,التمويل العمومي,-386.006,1.928
2016,وزارة الداخلية,0.0,383.300,نفقات التنمية,113.300,التمويل العمومي,-270.000,3.383
2019,مجلس نواب الشعب,0.0,29.793,NaN,30.923,نفقات التصرف,1.130,1.038


In [12]:
expenses.loc[
    (expenses.organization_name.str.strip() == "مجلس نواب الشعب")
    & (expenses.year == year)
    & (expenses.parent_name.str.strip() == "نفقات التصرف")
]

,parent_name,value,extra,year,organization_name,budget_type_name
411,نفقات التصرف,26.211,0.0,2019,مجلس نواب الشعب,التأجير العمومي
701,نفقات التصرف,3.582,0.0,2019,مجلس نواب الشعب,وسائل المصالح


## Compensation

In [13]:
comp_cols = ["ministry", "budget_type"]
comp_cols.extend(reversed(range(2015, 2020)))  # reversed because rtl
converters = {}
for year in range(2015, 2020):
    converters[year] = pd.to_numeric
comp = (
    pd.read_excel(
        "data/compensation.xlsx",
        sheet_name="الدعم ",
        skiprows=1,
        nrows=20,
        usecols=range(7),
        names=comp_cols,
        #converters=converters
    )
    .pipe(lambda df: df.assign(ministry=df.ministry.fillna(method="ffill")))
    .pipe(lambda df: df.loc[df.ministry.str.strip() != "الجملة"])
)

In [14]:
names = rt.name.unique()
#names = bt.name.unique()

In [15]:
for c in combinations(names, 2):
    r = fuzz.token_sort_ratio(*c)
    if 90 < r < 100:
        print(c)

('المداخيل الجبائية الاعتيادية ', 'المداخيل غير الجبائية الاعتيادية ')
('المداخيل الجبائية الاعتيادية ', 'المداخيل المالية الاعتيادية')
('مداخيل بعنوان القيمة الزائدة العقارية (I)', 'مداخيل بعنوان القيمة الزائدة العقارية (III)')
('الأشخاص المعنوين : الشركات البترولية ', 'الأشخاص المعنوين : الشركات غير البترولية')
('الأشخاص الطبيعيون (III)', 'الأشخاص الطبيعيون (IV)')
('الشركات البترولية (III)', 'الشركات البترولية (IV)')
('الشركات البترولية (III)', 'الشركات غير البترولية (III)')
('الشركات البترولية (IV)', 'الشركات غير البترولية (IV)')
('الشركات غير البترولية (III)', 'الشركات غير البترولية (IV)')
('مداخيل غير اعتيادية اخرى ', 'مداخيل غير اعتيادية اخرى  d')
('الموارد الجبائية الموضفة لصناديق الخزينة ', 'الموارد غير الجبائية الموظفة لصناديق الخزينة')


In [12]:
from utils import merge

In [13]:
revenues = merge(
    hierarchy_df=rt,
    hierarchy_df_on="name",
    values_df=rbt,
    values_df_on="resource_type_description",
    drop_cols=[
        "name_fr",
        "name_ar",
        "name_en",
        "description",
        "description_fr",
        "description_en",
        "description_ar",
        "level",
        "parent_id",
        "id_x",
        "id_y",
        "type",
        "resource_type_description",
    ],
)

In [14]:
summed = (
    revenues.groupby(["year", "parent_name"])
    .agg(sum)
    .reset_index()
    .rename(columns={"parent_name": "name"})
)
typed = revenues.loc[
    revenues.name.isin(revenues.parent_name), ["name", "year", "value"]
]

In [15]:
revenues_gap = pd.merge(
    summed, typed, on=["name", "year"], suffixes=("_summed", "_typed")
).pipe(lambda df: df.assign(gap=np.round(df.value_typed - df.value_summed, 3)))

In [16]:
revenues_gap.pipe(lambda df: df.loc[df.gap != 0]).sort_values(
    ["year", "name", "value_typed"]
).set_index("year")

,name,value_summed,value_typed,gap
year,,,,
2015,موارد ميزانية الدولة,7616.0,28900.0,21284.0
2016,الضريبة على دخل الأشخاص الطبيعين و الضريبة على...,1094.0,1103.0,9.0
2016,الضريبة على دخل الأشخاص الطبيعيين و الضريبة عل...,1195.0,1530.0,335.0
2016,المداخيل الجبائية الاعتيادية,11209.2,19987.2,8778.0
2016,موارد ميزانية الدولة,6974.0,29150.0,22176.0
2017,موارد ميزانية الدولة,8960.0,32200.0,23240.0
2018,المداخيل الجبائية الاعتيادية,14462.2,22847.2,8385.0
2018,المداخيل غير الجبائية الاعتيادية,946.0,1756.0,810.0
2018,موارد ميزانية الدولة,10431.0,35815.0,25384.0


In [39]:
for year in expenses.year.unique():
    for organization in expenses.loc[
        expenses.year == year, "organization_name"
    ].unique():
        res_df = expenses[
            (expenses.year == year) & (expenses.organization_name == organization)
        ]
        res_df.to_csv(
            "data/open/{}_{}.csv".format(organization.strip().replace(" ", "_"), year),
            index=False
        )
        res_df.to_json(
            "data/open/{}_{}.json".format(organization.strip().replace(" ", "_"), year),
            index=False,
            orient="table",
        )

In [18]:
expenses[expenses.parent_name.isna()]

,parent_name,value,extra,year,organization_name,budget_type_name
1460,NaN,380.000,0.0,2015,وزارة التكوين المهني و التشغيل,صناديق الخزينة
1461,NaN,3.000,0.0,2015,رئاسة الحكومة,صناديق الخزينة
1462,NaN,16.000,0.0,2015,وزارة أملاك الدولة والشؤون العقارية,صناديق الخزينة
1463,NaN,0.100,0.0,2015,وزارة الاقتصاد و المالية,صناديق الخزينة
1464,NaN,0.500,0.0,2015,وزارة التجارة و الصناعات التقليدية,صناديق الخزينة
1465,NaN,106.000,0.0,2015,وزارة التجهيز والتهيئة الترابية و التنمية المس...,صناديق الخزينة
1466,NaN,100.000,0.0,2015,وزارة التعليم العالي والبحث العلمي وتكنولوجيا ...,صناديق الخزينة
1467,NaN,4.000,0.0,2015,وزارة الثقافة,صناديق الخزينة
1468,NaN,109.200,0.0,2015,وزارة الداخلية,صناديق الخزينة
1469,NaN,13.000,0.0,2015,وزارة الدفاع الوطني,صناديق الخزينة
